## Identify tees and elbows using pipe centerlines

#### Setup

In [ ]:
# general
import json
import collections
import math
import uuid
import random
import pickle
import os
import time
from itertools import islice
import numpy as np
from tqdm.notebook import tqdm
import open3d as o3d

from src.geometry import sq_distance, vector_mag, sq_dist_vect, norm_array
from src.visualisation import *
from src.elements import *
from src.chamfer import *

In [ ]:
blueprint = 'data/sample.ifc'
data_path = "/mnt/c/data/3D_CAD/"


In [ ]:
# load data
with open(data_path + 'edges_westdeckbox.pkl', 'rb') as f:
    edges = pickle.load(f)
with open(data_path + 'nodes_westdeckbox.pkl', 'rb') as f:
    node_info = pickle.load(f)
    nodes = node_info[0]
    


In [ ]:
print(len(nodes), nodes[0])
print(len(edges), edges[0])

In [ ]:
# # create simplified dataset using subset
# #subset_size = 100
# subset_size = len(edges)
# edges_sub = edges[:subset_size]

# nodes_sub_ids = []
# for e in edges_sub:
#     nodes_sub_ids.append(e[0])
#     nodes_sub_ids.append(e[1])
# nodes_sub_ids = list(set(nodes_sub_ids))
# nodes_sub = [nodes[i] for i in nodes_sub_ids]

# print(len(nodes_sub), len(edges_sub), nodes_sub[0])

# nodes = nodes_sub
# edges = edges_sub

In [ ]:
e1 = [e[0] for e in edges]
e2 = [e[1] for e in edges]
print(max(e1), max(e2))
print(len(nodes))
print(len(edges))

#### Detect nearby edges

In [ ]:

# find edges of pipes
# format - category, centerpoint, bounding box, principal direction, element id
pipe_candidates = [[n[0], np.array(n[1]), np.array(n[2]), np.array(n[3]), n[4]] for n in nodes if n[0]==3]

# remove ridiculously large pipes
pipes = []
removed_pipes = []
for p in pipe_candidates:
    r = get_radius_from_bbox(p[2])
    if (r < ridiculously_large_pipe_threshold):
        pipes.append(p)
    else:
        removed_pipes.append(p)
        
pipe_edges = [(p[1] - (max(p[2])/2 * p[3]),
               p[1] + (max(p[2])/2 * p[3])) for p in pipes]

print(len(pipes))

In [ ]:
# compare edges to find close-by edges
count = 0
nearby_edges = []
for i, pe1 in tqdm(enumerate(pipe_edges)):
    for j, pe2 in enumerate(pipe_edges[i+1:]):
        
        # rough radius check to check compatibility
        if not radius_check([pipes[i],  pipes[j]], 0.2):
            continue
            
        # set dynamic threshold based on avg. radius
        r1 = get_radius_from_bbox(pipes[i][2])
        r2 = get_radius_from_bbox(pipes[j][2])
        dynamic_threshold = dist_threshold * (r1+r2)
        
        nearby_pair = edge_proximity_criteria(pe1, pe2, dynamic_threshold)
        if nearby_pair is not None:
            nearby_edges.append(((i,j+i+1), nearby_pair, count))
            count += 1
print(len(nearby_edges), nearby_edges[0])

In [ ]:
print(pipe_edges[0])
print(pipes[0])
lens = [max(p[2]) for p in pipes]
print(max(lens), min(lens), sum(lens)/len(lens))

#### Detect tees

In [ ]:
# rough check for tees
possible_tees = []
for i, ne1 in tqdm(enumerate(nearby_edges)):
    for j, ne2 in enumerate(nearby_edges[i+1:]):
        pt = edge_match(ne1, ne2)
        if pt is not None:
            possible_tees.append(pt)
            
print(len(possible_tees), possible_tees[0])

In [ ]:
# check for tees
# if two of the elements seem to be connected in a straight line and the other is placed like an elbow, its considered a tee
tee_connections = []
for i, pt in tqdm(enumerate(possible_tees)):
   
    # check for straight line connection
    pipe_connection = False
    if pipe_check(pipes[pt[0][0]], pipes[pt[0][1]]):
        if elbow_check(pipes[pt[0][2]], pipes[pt[0][0]]) and elbow_check(pipes[pt[0][2]], pipes[pt[0][0]]):
            pipe_connection = True
            pipe_pair = [0,1]
            other = 2
    if pipe_check(pipes[pt[0][0]], pipes[pt[0][2]]):
        if elbow_check(pipes[pt[0][1]], pipes[pt[0][0]]) and elbow_check(pipes[pt[0][1]], pipes[pt[0][2]]):       
            pipe_connection = True
            pipe_pair = [0,2]
            other = 1
    if pipe_check(pipes[pt[0][1]], pipes[pt[0][2]]):
        if elbow_check(pipes[pt[0][0]], pipes[pt[0][1]]) and elbow_check(pipes[pt[0][0]], pipes[pt[0][2]]):
            pipe_connection = True
            pipe_pair = [1,2]
            other = 0
        
    # check for angled connection
    if pipe_connection:
        if elbow_check(pipes[pt[0][pipe_pair[0]]], pipes[pt[0][other]]):
            tee_connections.append((pt, pipe_pair, other))
            
print(len(tee_connections), tee_connections[0])
        

#### Testing and visualisation

In [ ]:
tee_points, refined_tees = visualise_tees(tee_connections, blueprint, pipes, pipe_edges)
print(refined_tees[0])

In [ ]:
tee_points = o3d.utility.Vector3dVector(tee_points)
tee_cloud = o3d.geometry.PointCloud()
tee_cloud.points = tee_points
o3d.io.write_point_cloud("bp_tee_centerline_08.pcd", tee_cloud)


In [ ]:
pipe_points = visualise_pipes(pipes)

In [ ]:
pipe_points = o3d.utility.Vector3dVector(pipe_points)
pipe_cloud = o3d.geometry.PointCloud()
pipe_cloud.points = pipe_points
o3d.io.write_point_cloud("bp_pipe_centerline.pcd", pipe_cloud)


#### elbow test and vis

In [ ]:
# filter out detected tees from nearby edges to get elbow candidates
elbow_candidates = []
for ne in nearby_edges:
    edge_used = False
    for tee in refined_tees:
        if ne[2] in tee[0][2]:
            edge_used = True
            break
    if not edge_used:
        elbow_candidates.append(ne)
        
print(len(elbow_candidates), elbow_candidates[0])

In [ ]:
elbow_connections = []
for ec in elbow_candidates:
    if elbow_check(pipes[ec[0][0]], pipes[ec[0][1]], thresh=0.8, intersection_test=False):
        elbow_connections.append(ec)
        
print(len(elbow_connections), elbow_candidates[0])

In [ ]:
elbow_subset = elbow_connections
cl, ref_elbows = visualise_elbows(elbow_subset, pipes, pipe_edges)

# connected_pipes1 = [pipes[ec[0][0]] for ec in elbow_subset]
# connected_pipes2 = [pipes[ec[0][1]] for ec in elbow_subset]
# print(len(ref_elbows))
# connected_pipes = connected_pipes1 + connected_pipes2
# cl_p = visualise_pipes(connected_pipes)

# cl = np.concatenate([cl, cl_p])

In [ ]:
elbow_cloud = o3d.geometry.PointCloud()
elbow_points = o3d.utility.Vector3dVector(cl)
elbow_cloud.points = elbow_points
o3d.io.write_point_cloud("bp_elbow_centerline.pcd", elbow_cloud)


In [ ]:
f = open('refined_res.pkl', 'wb')
pickle.dump([refined_tees, ref_elbows], f)

### Metrics

In [ ]:
# identify true positive tees and elbows using graph edges
elbow_nodes = [i for i, n in enumerate(nodes) if n[0]==1]
tee_nodes = [i for i, n in enumerate(nodes) if n[0]==2]

# print(elbow_nodes[0])
# print(tee_nodes[0])

tp_tees, tp_elbows = [], []

# for tn in tee_nodes:
#     tee_edges = []
#     for e in edges:
#         # if the relationship contains the tee, add the other element to list of tee_edges, if it is a pipe
#         if tn == e[0]:
#             if nodes[e[1]][0] == 3:
#                 tee_edges.append(e[1])
#         elif tn == e[1]:
#             if nodes[e[0]][0] == 3:
#                 tee_edges.append(e[0])
#     tp_tees.append(tee_edges)

# l = [len(t) for t in tp_tees]
# print(l)

for en in elbow_nodes:
    elbow_edges = []
    for e in edges:
        # if the relationship contains the elbow, add the other element to list of elbow_edges, if it is a pipe
        if en == e[0]:
            if nodes[e[1]][0] == 3 or True:
                elbow_edges.append(e[1])
        elif en == e[1]:
            if nodes[e[0]][0] == 3 or True:
                elbow_edges.append(e[0])
    tp_elbows.append(elbow_edges)


l = [len(e) for e in tp_elbows]
print(l)


In [ ]:
print(sum(l), len(l))

In [ ]:
"""
solutions for lack of annotations: 
1. check for existence of elements instead connections - something like bounding box overlap
2. visualise the issue first using the ifc relationship viewer code - 1
3. find a way to manually annotate connections - at least tees

In [ ]:
print(nodes[edges[0][0]])

In [ ]:
# visualise the annotations to check for errors - code taken from graph experiments notebook TOFDO:organize code
ifc = ifcopenshell.open(data_path + "merged.ifc")

In [ ]:
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
owner_history = ifc.by_type("IfcOwnerHistory")[0]
project = ifc.by_type("IfcProject")[0]
context = ifc.by_type("IfcGeometricRepresentationContext")[0]
floor = ifc.by_type("IfcBuildingStorey")[0]

red = ifc.createIfcColourRgb('red', Red=0.9, Green=0.0, Blue=0.0)
green = ifc.createIfcColourRgb('green', Red=0.0, Green=0.9, Blue=0.0)
yellow = ifc.createIfcColourRgb('yellow', Red=0.9, Green=0.9, Blue=0.0)

# visualize results on ifc file
def draw_predictions(preds, nodes, ifc, colour):
    for pair in tqdm(preds):
        element1 = ifc.by_id(nodes[pair[0]][4])
        element1_name = element1.Name
        element2 = ifc.by_id(nodes[pair[1]][4])
        element2_name = element2.Name
        
        draw_relationship(element1_name, element1, element2_name, 
                          element2, ifc, floor, owner_history, context, colour)

In [ ]:
draw_predictions(edges, nodes, ifc, green)


In [ ]:
ifc.write('annotations_bp_vis_test.ifc')